In [12]:
import numpy as np
import os
import pandas as pd
import rpy2.robjects as robjects
import time

In [2]:
outpath = "output/interactions/"

# Get phenotypes with at least two significant associatons

In [3]:
# get list of significant HC phenotypes
f = open("output/compare_cutoff_pvals/sig_phe_HC.txt","r")
sig_phes = f.readline().split()
f.close()

# get list of significant cancer phenotypes
f = open("output/compare_cutoff_pvals/sig_phe_cancer.txt","r")
sig_phes += f.readline().split()
f.close()

len(sig_phes)

48

# Find which analyses have finished

In [34]:
fin_phes = []
for file in os.listdir("../scripts/output/interact/"):
    if file.endswith(".txt"):
        fin_phes.append(file.split("_")[0])
        #print(os.path.join("/mydir", file))
len(fin_phes)

47

# Find missing

In [35]:
set(sig_phes) - set(fin_phes)

{'HC303'}

# Get interaction p values

In [36]:
big_fat_df = pd.DataFrame(index = fin_phes, columns = ['allelotype_pair', 'estimate_add1', 'se_add1', 'z_add1', 'p_add1', 'estimate_add2', 'se_add2', 'z_add2', 'p_add2', 'estimate_int', 'se_int', 'z_int', 'p_int'])

In [1]:
t0 = time.time()
count = 1

def get_pval(split_line):
    p_val = split_line[1]
    if p_val == "<2e-16" or p_val == "<":
        p_val = 2e-16
    else:
        p_val = float(pval)
    return pval

def assign_estimate_se_z(split_line, big_fat_df, typestring):
    big_fat_df[phe]['estimate_' + typestring] = split_line[1]
    big_fat_df[phe]['se_' + typestring] = split_line[2]
    big_fat_df[phe]['z_' + typestring] = split_line[3]
    return big_fat_df

for file in os.listdir("../scripts/output/interact/"):
    if file.endswith(".txt"):
        phe = file.split("_")[0]
        f = open("../scripts/output/interact/" + file)
        print(count)
        count += 1
        print(file)
        print("time: {}\n".format(time.time() - t0))
        for line in f.readlines():
            split_line = line.split('\s+')
            if line[0] == "$":
                inter = line[2:-2]
                big_fat_df[phe]['allelotype_pair'] = inter
                p_line = False
                
            elif "Pr(>|z|)" in line:
                p_line = True                
                
            elif split_line[0] == "as.numeric(gcounts1)":
                if p_line:
                    big_fat_df[phe]['p_add1'] = get_pval(split_line)
                else:
                    big_fat_df = assign_estimate_se_z(split_line, big_fat_df, 'add_1')

            elif split_line[0] == "as.numeric(gcounts2)":
                if p_line:
                    big_fat_df[phe]['p_add2'] = get_pval(split_line)
                else:
                    big_fat_df = assign_estimate_se_z(split_line, big_fat_df, 'add_2')
                
            elif split_line[0] == "as.numeric(gcounts1):as.numeric(gcounts2)":
                if p_line:
                    big_fat_df[phe]['p_int'] = get_pval(split_line)
                else:
                    big_fat_df = assign_estimate_se_z(split_line, big_fat_df, 'int')            
                    p_line = False
        f.close()
print("time: {}".format(time.time() - t0))

IndentationError: expected an indented block (<ipython-input-1-e64e4451a154>, line 14)

In [38]:
big_fat_df.head()

array(['DRB1_404-DQB1_302', 'DRB1_404-DQA1_301', 'DQA1_501-DQB1_201', ...,
       'A_301-B_2705', 'A_301-C_102', 'A_301-DRB1_1303'], dtype=object)

In [39]:
big_fat_df.to_csv(outpath + "interact_pvals_guhan.csv")

# Adjust pvalues

In [40]:
thresh = 0.05
s = big_fat_df[['p_add1', 'p_add2', 'p_int']].as_matrix().shape
flat_df = big_fat_df[['p_add1', 'p_add2', 'p_int']].as_matrix().flatten()
total = flat_df.shape[0]
nom_p = robjects.FloatVector(flat_df)
rpadjust = robjects.r['p.adjust']
adj_p = np.array(rpadjust(nom_p, "BY"))
adj_p = np.reshape(adj_p,s)
# find the number of adjusted values below the threshold
below_thresh = (adj_p < thresh).sum()
adj_p_df = pd.DataFrame(adj_p, index=big_fat_df[['p_add1', 'p_add2', 'p_int']].index.values, columns=big_fat_df[['p_add1', 'p_add2', 'p_int']].columns.values)
print("number below threshold: {}".format(below_thresh))
adj_p_df.to_csv(outpath + "interact_pvals_adj.csv")
adj_p_df;

number below threshold: 191


/home/users/jolivier/.local/lib/python2.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in less
  # Remove the CWD from sys.path while we load stuff.


In [41]:
sig_df = adj_p_df.loc[(adj_p_df < thresh).sum(axis=1) > 0, :]
sig_df = sig_df[sig_df.columns[(sig_df< thresh).any()]]
print("number of phenotypes with significant entry: {}".format(sig_df.shape[0]))
print("number of allelotypes with significant entry: {}".format(sig_df.shape[1]))
sig_df.to_csv(outpath + "interact_pvals_adj_sig.csv")
sig_df;

number of phenotypes with significant entry: 14
number of allelotypes with significant entry: 159


In [42]:
sig_df[sig_df < thresh].count().sort_values(ascending = False).to_csv(outpath + "allele_sig_counts.csv")
T_sig_df = sig_df.transpose()
T_sig_df[T_sig_df < thresh].count().sort_values(ascending = False).to_csv(outpath + "phe_sig_counts.csv")